In [1]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
df_val = pd.read_csv("data/val.csv")

In [2]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [3]:
import pymorphy3

sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = pymorphy3.MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw] #долгая Каноническая форма слова (например, форма единственного числа, именительного падежа для существительных)
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [4]:
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np
#import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, Conv2D, GlobalMaxPool1D, concatenate, Flatten, add, MaxPool1D, RepeatVector
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Masking, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, TimeDistributed, AveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping

In [5]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [6]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train) #Текст в последовательности
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length) #Эта функция преобразует список (длиной num_samples) последовательностей (списков целых чисел) в двумерный массив Numpy 
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [7]:
y_train = df_train['class'].values
y_val = df_val['class'].values

### RNN Model

In [8]:
model_name = 'RNN'
model = Sequential()
model.add(
    Embedding(input_dim=word_count, #это размер словаря в текстовых данных
              input_length=training_length, #это длина входных последовательностей, которую вы бы определили для любого входного слоя модели Keras
              output_dim=30, #это размер векторного пространства, в которое будут встроены слова. Он определяет размер выходных векторов из этого слоя для каждого слова. Например, это может быть 32 или 100 или даже больше.
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) #Маскирует последовательность,используя значение маски для пропуска таймфреймов
model.add(SimpleRNN(32, recurrent_dropout=0.2, return_sequences=True)) #возвращать всю последовательность выходных данных для каждого элемента (по одному вектору на каждый шаг)
model.add(SimpleRNN(32, recurrent_dropout=0.2))#отбрасывание линейных значений повторов 0.2
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#early_stopping=EarlyStopping(monitor='val_loss') #Класс EarlyStopping Остановить обучение, когда отслеживаемая метрика перестанет улучшаться 


tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+ model_name, 
    write_graph=False, update_freq=100, profile_batch=0)

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/10
319/319 [==============================] - 126s 356ms/step - loss: 0.6417 - accuracy: 0.6030 - val_loss: 0.5189 - val_accuracy: 0.7336
Epoch 2/10
319/319 [==============================] - 194s 608ms/step - loss: 0.4104 - accuracy: 0.8191 - val_loss: 0.5099 - val_accuracy: 0.7517
Epoch 3/10
319/319 [==============================] - 163s 510ms/step - loss: 0.1921 - accuracy: 0.9283 - val_loss: 0.6558 - val_accuracy: 0.7382
Epoch 4/10
319/319 [==============================] - 131s 411ms/step - loss: 0.0982 - accuracy: 0.9655 - val_loss: 0.7996 - val_accuracy: 0.7349
Epoch 5/10
319/319 [==============================] - 193s 605ms/step - loss: 0.0623 - accuracy: 0.9786 - val_loss: 0.9854 - val_accuracy: 0.7252
Epoch 6/10
319/319 [==============================] - 178s 559ms/step - loss: 0.0428 - accuracy: 0.9855 - val_loss: 1.0549 - val_accuracy: 0.7268
Epoch 7/10
319/319 [==============================] - 118s 371ms/step - loss: 0.0329 - accuracy: 0.9890 - val_loss: 1.1280 -

In [9]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 5s 100ms/step - loss: 1.4475 - accuracy: 0.7006


Test score: 1.4474695920944214
Test accuracy: 0.7005687355995178


### CNN -> RNN Model


In [10]:
model_name = 'CNN -> RNN'
model = Sequential()

model.add(
    Embedding(input_dim=word_count, #это размер словаря в текстовых данных
              input_length=training_length, #это длина входных последовательностей, которую вы бы определили для любого входного слоя модели Keras
              output_dim=64, #это размер векторного пространства, в которое будут встроены слова. Он определяет размер выходных векторов из этого слоя для каждого слова. Например, это может быть 32 или 100 или даже больше.
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) #Маскирует последовательность,используя значение маски для пропуска таймфреймов
model.add(Conv1D(128, 3, activation='relu', padding="same"))
model.add(SimpleRNN(64, recurrent_dropout=0.2)) #отбрасывание линейных значений повторов 0.2
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])




In [11]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/10
319/319 [==============================] - 270s 800ms/step - loss: 0.5938 - accuracy: 0.6573 - val_loss: 0.5018 - val_accuracy: 0.7520
Epoch 2/10
319/319 [==============================] - 185s 579ms/step - loss: 0.3443 - accuracy: 0.8544 - val_loss: 0.5392 - val_accuracy: 0.7470
Epoch 3/10
319/319 [==============================] - 190s 594ms/step - loss: 0.1416 - accuracy: 0.9479 - val_loss: 0.7532 - val_accuracy: 0.7368
Epoch 4/10
319/319 [==============================] - 174s 544ms/step - loss: 0.0646 - accuracy: 0.9773 - val_loss: 0.9039 - val_accuracy: 0.7302
Epoch 5/10
319/319 [==============================] - 157s 491ms/step - loss: 0.0339 - accuracy: 0.9883 - val_loss: 1.2097 - val_accuracy: 0.7297
Epoch 6/10
319/319 [==============================] - 148s 465ms/step - loss: 0.0191 - accuracy: 0.9938 - val_loss: 1.3958 - val_accuracy: 0.7219
Epoch 7/10
319/319 [==============================] - 150s 471ms/step - loss: 0.0129 - accuracy: 0.9957 - val_loss: 1.5766 -

### RNN -> CNN Model

In [12]:
model_name = 'RNN -> CNN'
model = Sequential()

model.add(
    Embedding(input_dim=word_count, #это размер словаря в текстовых данных
              input_length=training_length, #это длина входных последовательностей, которую вы бы определили для любого входного слоя модели Keras
              output_dim=64, #это размер векторного пространства, в которое будут встроены слова. Он определяет размер выходных векторов из этого слоя для каждого слова. Например, это может быть 32 или 100 или даже больше.
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) #Маскирует последовательность,используя значение маски для пропуска таймфреймов
model.add(SimpleRNN(64, recurrent_dropout=0.2, return_sequences=True)) #отбрасывание линейных значений повторов 0.2
# model.add(RepeatVector(32))
model.add(Conv1D(128, 3, activation='relu', padding="same"))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [13]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/10
319/319 [==============================] - 398s 1s/step - loss: 0.6309 - accuracy: 0.6058 - val_loss: 0.5134 - val_accuracy: 0.7423
Epoch 2/10
319/319 [==============================] - 381s 1s/step - loss: 0.4022 - accuracy: 0.8280 - val_loss: 0.4972 - val_accuracy: 0.7514
Epoch 3/10
319/319 [==============================] - 410s 1s/step - loss: 0.1681 - accuracy: 0.9406 - val_loss: 0.6628 - val_accuracy: 0.7328
Epoch 4/10
319/319 [==============================] - 369s 1s/step - loss: 0.0687 - accuracy: 0.9787 - val_loss: 0.9156 - val_accuracy: 0.7284
Epoch 5/10
319/319 [==============================] - 320s 1s/step - loss: 0.0337 - accuracy: 0.9893 - val_loss: 1.3654 - val_accuracy: 0.7271
Epoch 6/10
319/319 [==============================] - 232s 728ms/step - loss: 0.0222 - accuracy: 0.9934 - val_loss: 1.6010 - val_accuracy: 0.7240
Epoch 7/10
319/319 [==============================] - 224s 701ms/step - loss: 0.0163 - accuracy: 0.9951 - val_loss: 1.8883 - val_accuracy: 

#LSTM Models

In [14]:
model_name = 'LSTM'
model = Sequential()

model.add(
    Embedding(input_dim=word_count, #это размер словаря в текстовых данных
              input_length=training_length, #это длина входных последовательностей, которую вы бы определили для любого входного слоя модели Keras
              output_dim=30, #это размер векторного пространства, в которое будут встроены слова. Он определяет размер выходных векторов из этого слоя для каждого слова. Например, это может быть 32 или 100 или даже больше.
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) #Маскирует последовательность,используя значение маски для пропуска таймфреймов
model.add(LSTM(64, recurrent_dropout=0.2)) #отбрасывание линейных значений повторов 0.2
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/10
319/319 [==============================] - 269s 786ms/step - loss: 0.5536 - accuracy: 0.7081 - val_loss: 0.4974 - val_accuracy: 0.7537
Epoch 2/10
319/319 [==============================] - 206s 646ms/step - loss: 0.3229 - accuracy: 0.8669 - val_loss: 0.5343 - val_accuracy: 0.7478
Epoch 3/10
319/319 [==============================] - 203s 637ms/step - loss: 0.1538 - accuracy: 0.9427 - val_loss: 0.6857 - val_accuracy: 0.7377
Epoch 4/10
319/319 [==============================] - 244s 764ms/step - loss: 0.0898 - accuracy: 0.9676 - val_loss: 0.8755 - val_accuracy: 0.7325
Epoch 5/10
319/319 [==============================] - 242s 758ms/step - loss: 0.0634 - accuracy: 0.9772 - val_loss: 0.9870 - val_accuracy: 0.7298
Epoch 6/10
319/319 [==============================] - 246s 771ms/step - loss: 0.0473 - accuracy: 0.9825 - val_loss: 1.1033 - val_accuracy: 0.7292
Epoch 7/10
319/319 [==============================] - 268s 840ms/step - loss: 0.0371 - accuracy: 0.9861 - val_loss: 1.3783 -

In [15]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 5s 104ms/step - loss: 1.6871 - accuracy: 0.7082


Test score: 1.687084674835205
Test accuracy: 0.7082396745681763


#### Получим предсказание тональности твита

In [16]:
df_test.text.values

array(['тектоника рельефсамый ужасный мир мучение',
       'ходить запускать шар желание насна получиться хрен они',
       'хотеть лето ради направить ноготь яркий лак', ...,
       'rt killgayslut yournovocaine привеееть муд черта сэмми',
       'настроение вроде нормальный плакать хотеться',
       'зайти сон девчонкампока мыть посудунастя фоткаться httptcokinexudtuh'],
      dtype=object)

In [17]:
#предобработка текста для предсказания модели
def preprocess_text_for_prediction(txt):
  txt = preprocess_text(txt)
  txt = tokenizer.texts_to_sequences([txt])
  matr_txt = pad_sequences(txt, maxlen=training_length)
  mart_txt = matr_txt.reshape(1, -1)
  return matr_txt


In [18]:
model.predict(
    [preprocess_text_for_prediction('тектоника рельефсамый ужасный мир мучение')],
    batch_size=None,
    )

1/1 [==============================] - 5s 5s/step


array([[5.390437e-05]], dtype=float32)

In [19]:
model.predict(
    [preprocess_text_for_prediction('Люблю смотреть на звезды')],
    batch_size=None,
)

1/1 [==============================] - 0s 58ms/step


array([[0.0178717]], dtype=float32)

In [20]:
model.predict(
    [preprocess_text_for_prediction('Великолепно!')], 
    batch_size=None,
)

1/1 [==============================] - 0s 43ms/step


array([[0.9954847]], dtype=float32)

Вроде что-то определяет 

### Bidirectional LSTM

In [21]:
model_name = 'Bidirectional LSTM'
inputs = Input(shape=(X_train.shape[1],))

x =     Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True)(inputs)

xbi = Bidirectional(LSTM(15, return_sequences=True))(x) #Двунаправленный
x = add([x, xbi])

x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)



model = Model(inputs=inputs, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:

history_1 = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/10
319/319 [==============================] - 292s 740ms/step - loss: 0.5524 - accuracy: 0.7044 - val_loss: 0.4875 - val_accuracy: 0.7592
Epoch 2/10
319/319 [==============================] - 189s 593ms/step - loss: 0.2940 - accuracy: 0.8801 - val_loss: 0.5426 - val_accuracy: 0.7534
Epoch 3/10
319/319 [==============================] - 204s 639ms/step - loss: 0.1194 - accuracy: 0.9571 - val_loss: 0.7136 - val_accuracy: 0.7386
Epoch 4/10
319/319 [==============================] - 180s 564ms/step - loss: 0.0633 - accuracy: 0.9782 - val_loss: 0.8822 - val_accuracy: 0.7301
Epoch 5/10
319/319 [==============================] - 191s 599ms/step - loss: 0.0394 - accuracy: 0.9868 - val_loss: 1.0846 - val_accuracy: 0.7274
Epoch 6/10
319/319 [==============================] - 186s 584ms/step - loss: 0.0258 - accuracy: 0.9912 - val_loss: 1.1997 - val_accuracy: 0.7225
Epoch 7/10
319/319 [==============================] - 152s 474ms/step - loss: 0.0185 - accuracy: 0.9939 - val_loss: 1.3770 -

In [23]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 9s 192ms/step - loss: 1.7228 - accuracy: 0.7105


Test score: 1.7227580547332764
Test accuracy: 0.7104880213737488


#CNN Model


In [24]:
model_name = 'CNN'
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=100,
              trainable=True,
              mask_zero=True))
#model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [25]:

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/10
319/319 [==============================] - 237s 720ms/step - loss: 0.5384 - accuracy: 0.7163 - val_loss: 0.4818 - val_accuracy: 0.7601
Epoch 2/10
319/319 [==============================] - 225s 705ms/step - loss: 0.2524 - accuracy: 0.8978 - val_loss: 0.5532 - val_accuracy: 0.7473
Epoch 3/10
319/319 [==============================] - 212s 664ms/step - loss: 0.0664 - accuracy: 0.9769 - val_loss: 0.7382 - val_accuracy: 0.7376
Epoch 4/10
319/319 [==============================] - 191s 599ms/step - loss: 0.0195 - accuracy: 0.9941 - val_loss: 0.9164 - val_accuracy: 0.7369
Epoch 5/10
319/319 [==============================] - 193s 606ms/step - loss: 0.0061 - accuracy: 0.9985 - val_loss: 1.0737 - val_accuracy: 0.7349
Epoch 6/10
319/319 [==============================] - 212s 664ms/step - loss: 0.0018 - accuracy: 0.9997 - val_loss: 1.1917 - val_accuracy: 0.7371
Epoch 7/10
319/319 [==============================] - 221s 692ms/step - loss: 7.1542e-04 - accuracy: 0.9999 - val_loss: 1.28

In [26]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 3s 70ms/step - loss: 1.7641 - accuracy: 0.7192


Test score: 1.764114499092102
Test accuracy: 0.7191729545593262


#CNN -> LSTM

In [27]:
model_name = 'CNN'
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3, activation='relu', padding="same"))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/10
319/319 [==============================] - 412s 1s/step - loss: 0.5475 - accuracy: 0.7111 - val_loss: 0.4865 - val_accuracy: 0.7577
Epoch 2/10
319/319 [==============================] - 325s 1s/step - loss: 0.2979 - accuracy: 0.8770 - val_loss: 0.5426 - val_accuracy: 0.7425
Epoch 3/10
319/319 [==============================] - 315s 986ms/step - loss: 0.1360 - accuracy: 0.9498 - val_loss: 0.7235 - val_accuracy: 0.7362
Epoch 4/10
319/319 [==============================] - 346s 1s/step - loss: 0.0758 - accuracy: 0.9718 - val_loss: 0.8895 - val_accuracy: 0.7329
Epoch 5/10
319/319 [==============================] - 372s 1s/step - loss: 0.0458 - accuracy: 0.9833 - val_loss: 1.2123 - val_accuracy: 0.7311
Epoch 6/10
319/319 [==============================] - 414s 1s/step - loss: 0.0299 - accuracy: 0.9893 - val_loss: 1.3137 - val_accuracy: 0.7270
Epoch 7/10
319/319 [==============================] - 395s 1s/step - loss: 0.0218 - accuracy: 0.9922 - val_loss: 1.4388 - val_accuracy: 0.7

In [28]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

 8/45 [====>.........................] - ETA: 8s - loss: 2.1082 - accuracy: 0.7075

45/45 [==============================] - 9s 191ms/step - loss: 2.0548 - accuracy: 0.7151


Test score: 2.05476975440979
Test accuracy: 0.7151170372962952


#LSTM -> CNN

In [29]:
model_name = 'LSTM -> CNN'
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, return_sequences=True))
model.add(Conv1D(64, 3, activation='relu', padding="same"))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# early_stopping=EarlyStopping(monitor='val_loss')  

# tensorboard_callback = tf.keras.callbacks.TensorBoard(
#     log_dir='logs/'+ model_name, 
#     write_graph=False, update_freq=100, profile_batch=0)

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1)#,
                    #callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10
319/319 [==============================] - 319s 953ms/step - loss: 0.5530 - accuracy: 0.7059 - val_loss: 0.4867 - val_accuracy: 0.7597
Epoch 2/10
319/319 [==============================] - 266s 834ms/step - loss: 0.3091 - accuracy: 0.8724 - val_loss: 0.5460 - val_accuracy: 0.7433
Epoch 3/10
319/319 [==============================] - 237s 743ms/step - loss: 0.1473 - accuracy: 0.9454 - val_loss: 0.6929 - val_accuracy: 0.7339
Epoch 4/10
319/319 [==============================] - 242s 759ms/step - loss: 0.0889 - accuracy: 0.9685 - val_loss: 0.8381 - val_accuracy: 0.7304
Epoch 5/10
319/319 [==============================] - 234s 735ms/step - loss: 0.0590 - accuracy: 0.9786 - val_loss: 1.1037 - val_accuracy: 0.7233
Epoch 6/10
319/319 [==============================] - 237s 743ms/step - loss: 0.0427 - accuracy: 0.9841 - val_loss: 1.2339 - val_accuracy: 0.7266
Epoch 7/10
319/319 [==============================] - 242s 759ms/step - loss: 0.0324 - accuracy: 0.9876 - val_loss: 1.3958 -

In [30]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 6s 132ms/step - loss: 1.9735 - accuracy: 0.7129


Test score: 1.9734731912612915
Test accuracy: 0.7129127383232117


Все модели показали относительно одинаковые результаты, похоже что сети переобучаются.